In [ ]:
import cv2
import numpy as np
from PIL import Image
import os
import imageio_ffmpeg
import subprocess

In [ ]:
# File paths, loop or proceed one by one for all seasons
png_path = 'Input/Frame_PNG/PNG_Frame_Winter.png'
video_path = 'Output/Wind_Map_Winter.mp4'
output_path = 'Output/Wind_Map_Winter.avi' 

In [ ]:
# Load the transparent PNG
overlay_rgba = Image.open(png_path).convert("RGBA")

# Open input video
cap = cv2.VideoCapture(video_path)
fps = cap.get(cv2.CAP_PROP_FPS)
width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

# Verify PNG matches video
if overlay_rgba.size != (width, height):
    raise ValueError(f"Overlay size {overlay_rgba.size} doesn't match video size {(width, height)}")

In [ ]:
# Use MJPEG codec for best compatibility
fourcc = cv2.VideoWriter_fourcc(*'MJPG')
out = cv2.VideoWriter(output_path, fourcc, fps, (width, height))

# Frame loop
while True:
    ret, frame = cap.read()
    if not ret:
        break

    # Convert frame to RGBA with PIL
    frame_rgba = Image.fromarray(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)).convert("RGBA")

    # Composite PNG overlay
    frame_rgba.alpha_composite(overlay_rgba)

    # Convert back to OpenCV BGR
    final = cv2.cvtColor(np.array(frame_rgba.convert("RGB")), cv2.COLOR_RGB2BGR)
    out.write(final)

# Clean up
cap.release()
out.release()

# File info
print("Video saved:", output_path)
print("Size:", round(os.path.getsize(output_path) / (1024 * 1024), 2), "MB")

In [ ]:
# Input video paths
video_paths = [
    'Output/Wind_Map_Spring_Framed.avi',
    'Output/Wind_Map_Summer_Framed.avi',
    'Output/Wind_Map_Autumn_Framed.avi',
    'Output/Wind_Map_Winter_Framed.avi'
]

# Output paths
output_path = "Output/Wind_Map_4Seasons.mp4"
temp_avi = "Output/temp_uncompressed.avi"

# Constants for duration and transition control
fps = 24
fade_frames = int(0.5 * fps)
clip_frames = int(5.5 * fps)

In [ ]:
# Load trimmed video segment
def load_trimmed_video(path, target_frames):
    cap = cv2.VideoCapture(path)
    total = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    print(f"Loading {path} (total frames: {total})")

    frames = []
    start = max(0, (total - target_frames) // 2)
    cap.set(cv2.CAP_PROP_POS_FRAMES, start)

    while len(frames) < target_frames:
        ret, frame = cap.read()
        if not ret:
            break
        frames.append(frame)
    cap.release()

    if len(frames) < target_frames:
        raise ValueError(f"Video {path} has too few frames.")
    print(f"Loaded {len(frames)} frames from {path}")
    return frames

# Build final sequence with crossfades
final_frames = []
width, height = None, None

for idx, path in enumerate(video_paths):
    frames = load_trimmed_video(path, clip_frames)

    if width is None or height is None:
        height, width = frames[0].shape[:2]

    if idx == 0:
        final_frames.extend(frames[:-fade_frames])
    else:
        print(f"Creating crossfade from {video_paths[idx - 1]} → {path}")
        for i in range(fade_frames):
            alpha = i / fade_frames
            blended = cv2.addWeighted(prev_tail[i], 1 - alpha, frames[i], alpha, 0)
            final_frames.append(blended)
        final_frames.extend(frames[fade_frames:-fade_frames])

    prev_tail = frames[-fade_frames:]

final_frames.extend(prev_tail)

print("Total frames collected:", len(final_frames))

In [ ]:
# Write to temp AVI
fourcc = cv2.VideoWriter_fourcc(*'MJPG')
writer = cv2.VideoWriter(temp_avi, fourcc, fps, (width, height))

print("Writing temp AVI...")
for i, frame in enumerate(final_frames):
    writer.write(frame)
    if i % 100 == 0 or i == len(final_frames) - 1:
        print(f"  Wrote frame {i + 1} / {len(final_frames)}")
writer.release()

if not os.path.exists(temp_avi):
    raise RuntimeError("Temp AVI was not created.")
else:
    print("Temp AVI created:", temp_avi)
    print("Temp AVI size:", round(os.path.getsize(temp_avi) / (1024 * 1024), 2), "MB")

In [ ]:
# Convert to MP4
ffmpeg_cmd = [
    imageio_ffmpeg.get_ffmpeg_exe(),
    '-y',
    '-i', temp_avi,
    '-vcodec', 'libx264',
    '-pix_fmt', 'yuv420p',
    '-crf', '18',
    output_path
]

print("Encoding to final MP4...")
result = subprocess.run(ffmpeg_cmd, capture_output=True, text=True)

if result.returncode != 0:
    print("FFmpeg failed:")
    print(result.stderr)
else:
    print("Final MP4 created:", output_path)
    print("Final size:", round(os.path.getsize(output_path) / (1024 * 1024), 2), "MB")
    print("Final duration (s):", round(len(final_frames) / fps, 2))